In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [5]:
import json
from satori.rtm.client import make_client, SubscriptionMode

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=10   # number of inputs
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    lastPrice=float(json.loads(json.dumps(messagebox.pop(), ensure_ascii=False))['price']) # get that first price
    proxy=0
    j=1 # j = nombre de rendements
    proxies=np.empty(0) # x of the RLS algorithm
    proxies=np.append(proxies,1)
    while proxies.size<n:            
        if messagebox:
            message=messagebox.pop()
            message=json.dumps(message, ensure_ascii=False)
            message=json.loads(message)
            proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
            lastPrice=float(message['price'])
            proxies=np.append(proxies, proxy)
            j+=1
            print(proxies, proxies.size)
        time.sleep(1)
    y=np.sum(proxies)+0.1*np.random.rand(1)[0]
    message=np.array2string(np.append([i,y], proxies),separator=",") 
    producer.send('test2', message)
    i=i+1
    print(message)
    while True:
        proxies=np.roll(proxies,-1)
        proxies[0]=1
        while not messagebox:
            time.sleep(1)
        message=messagebox.pop()
        message=json.dumps(message, ensure_ascii=False)
        message=json.loads(message)
        proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
        lastPrice=float(message['price'])
        proxies[n-1]=proxy
        j+=1
        y=np.sum(proxies)+0.1*np.random.rand(1)[0]
        message=np.array2string(np.append([i,y], proxies),separator=",") 
        producer.send('test2', message)
        i=i+1
        print(message)

Connected to Satori RTM!
(array([  1.00000000e+00,   2.02646572e-05]), 2)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05]), 3)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05]), 4)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05,   3.39927872e-05]), 5)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05,   3.39927872e-05,   3.47421387e-05]), 6)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05,   3.39927872e-05,   3.47421387e-05,
         3.50472567e-05]), 7)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05,   3.39927872e-05,   3.47421387e-05,
         3.50472567e-05,   3.64944956e-05]), 8)
(array([  1.00000000e+00,   2.02646572e-05,   3.17115195e-05,
         3.34703254e-05,   3.39927872e-05,   3.47421387e-05,
         3.50472567e-05,   3.64944956e-05,   3.70127612e-05]), 9)
(array([  1.0000

[  3.40000000e+01,  1.06690031e+00,  1.00000000e+00,  4.71643345e-05,
   4.74484542e-05,  4.76725162e-05,  4.77183237e-05,  4.77311271e-05,
   4.77384583e-05,  4.81314814e-05,  4.81632547e-05,  4.82187239e-05]
[  3.50000000e+01,  1.01175762e+00,  1.00000000e+00,  4.74484542e-05,
   4.76725162e-05,  4.77183237e-05,  4.77311271e-05,  4.77384583e-05,
   4.81314814e-05,  4.81632547e-05,  4.82187239e-05,  4.82589185e-05]
[  3.60000000e+01,  1.05831921e+00,  1.00000000e+00,  4.76725162e-05,
   4.77183237e-05,  4.77311271e-05,  4.77384583e-05,  4.81314814e-05,
   4.81632547e-05,  4.82187239e-05,  4.82589185e-05,  4.82594373e-05]
[  3.70000000e+01,  1.09351741e+00,  1.00000000e+00,  4.77183237e-05,
   4.77311271e-05,  4.77384583e-05,  4.81314814e-05,  4.81632547e-05,
   4.82187239e-05,  4.82589185e-05,  4.82594373e-05,  4.83700566e-05]
[  3.80000000e+01,  1.08290593e+00,  1.00000000e+00,  4.77311271e-05,
   4.77384583e-05,  4.81314814e-05,  4.81632547e-05,  4.82187239e-05,
   4.82589185e-05,  

KeyboardInterrupt: 